## 03 Starnet predictions

The stardist model is used to create instance masks for a subsequent watershed. The main drawback of stardist is that one doesn't get accurate cell borders due to the limited number of polygons (depending on computational expenses). Therefore, this model 'only' operates with 32 angles.

In [ ]:
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import skimage
import stardist.models

import utils.evaluation
import utils.model_builder
import utils.visualize

### Import images

For testing, the BBBC006v1 image set from the Broad Bioimage Benchmark Collection was used [Ljosa et al., Nature Methods, 2012].

In [ ]:
root = './data/test_new/'

X = sorted(glob.glob(f'{root}/images/*.tif'))
Y = sorted(glob.glob(f'{root}/masks/*.tif'))

X = list(map(skimage.io.imread, X))
Y = list(map(skimage.io.imread, Y))

### Import models

In [ ]:
# UNet
model_unet = utils.model_builder.standard_unet()
model_unet.load_weights('./models/UNet.h5')

In [ ]:
# Stardist
model_star = stardist.models.StarDist2D(None, name='Stardist', basedir='./models/')

### Otsu

In [ ]:
utils.visualize.otsu_example(X, Y, ix=9)

In [ ]:
preds_otsu = utils.predictions.otsu_batch(X)
utils.evaluation.measure_iou(Y, preds_otsu, 'otsu')

### UNet

In [ ]:
utils.visualize.unet_example(model_unet, X, Y)

In [ ]:
preds_unet = utils.predictions.unet_batch(model_unet, X)
preds_unet_meas = [1-p[:,:,0] for p in preds_unet]
utils.evaluation.measure_iou(Y, preds_unet_meas, 'unet')

### Stardist

In [ ]:
utils.visualize.stardist_example(model_star, X, Y)

In [ ]:
preds_star = utils.predictions.stardist_batch(model_star, X)
utils.evaluation.measure_iou(Y, preds_star, 'stardist', prob=0)

### Starnet

In [ ]:
utils.visualize.starnet_example(model_star, model_unet, X, Y)

In [ ]:
preds_starnet = utils.predictions.starnet_batch(model_star, model_unet, X)
utils.evaluation.measure_iou(Y, preds_starnet, 'starnet', prob=0)